We'll use OCR code from here: https://towardsdatascience.com/optical-character-recognition-ocr-with-less-than-12-lines-of-code-using-python-48404218cccb

And the video code here: https://stackoverflow.com/a/29317298

In [4]:
import os
import cv2
import pytesseract
from uuid import uuid4
import numpy as np
import ffmpeg

In [5]:
OUT_DIR = '../data/processed/videos'
IN_DIR = '../data/raw/videos'

The regular OCR for tesseract is not fast enough. We'll need to use the fast langpacks

In [62]:
cap = cv2.VideoCapture('./data/raw/videos/oregon_cut.avi')
got_frame, _ = cap.read()
if not got_frame:
    raise Exception('No frames :(')

previous = ""
writer = None

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count = 0
while True:
    got_frame, frame = cap.read()

    # Crop the frame to just the text
    cropped = frame[910:1020, 910:950]

    # Threshold code from the article
    gray = cv2.cvtColor(cropped, cv2.COLOR_RGB2GRAY)
    gray, img_bin = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    gray = cv2.bitwise_not(img_bin)

    s = pytesseract.image_to_string(gray, lang='eng')

    if previous == "30" and s == "29":
        # match start
        size = (frame.shape[1], frame.shape[0])
        writer = cv2.VideoWriter(f'{OUT_DIR}/match-{uuid4()}.avi', -1, 20.0, size)

    corner_pixel = cropped[0, 0]
    background_red = corner_pixel[0] < 120 and corner_pixel[1] < 120 and corner_pixel[2] > 240
    if previous == "1" and s == "0" and background_red:
        # match end
        writer.release()
        writer = None

    if writer is not None:
        writer.write(frame)

    previous = s
    print(f'{frame_count}/{total_frames}; Detected: {s};', end='\r')
    frame_count += 1

cap.release()
cv2.destroyAllWindows()

0/1024516; Detected: ; Full: 0.07024538399127778; Tess: 0.06770792699535377
1/1024516; Detected: ; Full: 0.06468236799992155; Tess: 0.061687929002800956
2/1024516; Detected: ; Full: 0.0627590469957795; Tess: 0.060451965997344814
3/1024516; Detected: ; Full: 0.06462485699739773; Tess: 0.062232173993834294
4/1024516; Detected: ; Full: 0.06839558998763096; Tess: 0.06549803100642748
5/1024516; Detected: ; Full: 0.06666432300698943; Tess: 0.06430992898822296
6/1024516; Detected: ; Full: 0.06641132499498781; Tess: 0.06347690398979466
7/1024516; Detected: ; Full: 0.0658785940031521; Tess: 0.06294025700481143
8/1024516; Detected: ; Full: 0.06812489799631294; Tess: 0.06563253799686208
9/1024516; Detected: ; Full: 0.06685079100134317; Tess: 0.0644605059933383
10/1024516; Detected: ; Full: 0.08299358400108758; Tess: 0.07975492600235157
11/1024516; Detected: ; Full: 0.07895049800572451; Tess: 0.07339732500258833
12/1024516; Detected: ; Full: 0.073787193003227; Tess: 0.06942902800801676
13/1024516;

KeyboardInterrupt: 

Unfortunately, tesseract is just way too slow. Even using the fast language packs, it did considerably slower than real time

In [8]:
cap = cv2.VideoCapture('./data/raw/videos/oregon_cut.avi')
got_frame, _ = cap.read()
if not got_frame:
    raise Exception('No frames :(')

prev = (0, 0, 0)
recording = False

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count = 0

matches = []


def in_range(p, r):
    return r[0] + 10 >= p[0] >= r[0] - 10 and r[1] + 10 >= p[1] >= r[1] - 10 and r[2] + 10 >= p[2] >= r[2] - 10


while True:
    got_frame, frame = cap.read()

    if not got_frame:
        break

    detection_range = frame[928:948, 739:741]
    detection_range_avg = np.average(detection_range, axis=(0, 1))

    if not recording and in_range(prev, (150, 150, 150)) and not in_range(detection_range_avg, (150, 150, 150)):
        # match start
        recording = True
        matches.append([frame_count])

    if recording and in_range(detection_range_avg, (10, 10, 220)):
        # match end
        recording = False
        matches[-1].append(frame_count)

    frame_count += 1
    prev = detection_range_avg
    print(f'{frame_count / total_frames * 100:.2f}%', end='\r')

print(matches)

cap.release()
cv2.destroyAllWindows()

[[378, 5088], [10955, 15863], [16523, 26753], [33613, 38322], [41886, 46597], [59465, 64175], [65940, 70650], [80837, 85582], [87157, 91902], [92605, 92955], [93148, 101552], [102171, 106938], [107030, 112579], [125080, 129911], [133268, 138194], [143922, 148827], [152122, 157058], [168384, 173359], [175571, 180284], [183898, 188610], [189858, 196005], [196292, 202715], [203315, 208779], [217774, 223977], [225182, 230006], [240504, 245219], [246157, 250866], [257941, 262809], [265549, 280848], [283047, 287757], [291353, 296374], [301092, 305805], [306442, 308396], [308781, 315294], [328140, 333171], [338170, 342982], [343842, 350561], [355492, 360330], [363250, 368235], [369088, 378727], [510880, 516836], [519601, 524344], [528426, 549749], [553395, 558428], [560547, 566083], [570121, 575170], [577584, 584447], [643751, 709692], [715464, 720600], [749798, 755062], [783836, 807611], [824536, 843564], [873375, 878086]]


As seen above I inspect the progress bar. Looking at one pixel turns out to be too unpredictable so I look at an average. Even that has some issues because initially there is some grey left in the progress bar before going full green, so instead I'm just gonna look at shifts away from gray

In [13]:
in_file = ffmpeg.input('./data/raw/videos/oregon_cut.avi')
for (start, end) in matches:
    in_file
    .trim(start_frame=start, end_frame=end)
    .setpts('PTS-STARTPTS')
    .output(f'{OUT_DIR}/match-oregon-{start}-{end}.avi')
    .run()

ffmpeg version 5.0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 12 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-D_FORTIFY_SOURCE=2 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -Wl,--build-id=sha1 ' --extra-cflags=' -I/usr/include/rav1e' --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libvo-amrwbenc --enable-version3 --enable-bzlib --enable-chro

KeyboardInterrupt: 

In [ ]:
def in_range(p, r):
    return r[0] + 10 >= p[0] >= r[0] - 10 and r[1] + 10 >= p[1] >= r[1] - 10 and r[2] + 10 >= p[2] >= r[2] - 10


def has_text(cropped, text):
    gray = cv2.cvtColor(cropped, cv2.COLOR_RGB2GRAY)
    gray, img_bin = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    gray = cv2.bitwise_not(img_bin)
    return text == pytesseract.image_to_string(gray, lang='eng')


for video in os.listdir(IN_DIR):
    cap = cv2.VideoCapture(f'{IN_DIR}/{video}')

    got_frame, _ = cap.read()
    if not got_frame:
        print(f'Skipping {video}')

    prev = (0, 0, 0)
    recording = False

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_count = 0

    matches = []

    while True:
        got_frame, frame = cap.read()

        if not got_frame:
            break

        detection_range = frame[928:948, 739:741]
        text_range = frame[910:1020, 910:950]
        detection_range_avg = np.average(detection_range, axis=(0, 1))

        if not recording \
                and in_range(prev, (150, 150, 150)) \
                and not in_range(detection_range_avg, (150, 150, 150)) \
                and has_text(text_range, "29"):
            # match start
            recording = True
            matches.append([frame_count])

        if recording \
                and in_range(detection_range_avg, (10, 10, 220)) \
                and has_text(text_range, "0"):
            # match end
            recording = False
            matches[-1].append(frame_count)

        frame_count += 1
        prev = detection_range_avg
        print(f'{frame_count / total_frames * 100:.2f}%', end='\r')

    print(matches)

cap.release()